In [1]:
from robotic import ry
import numpy as np
import time

###########################################PART-2 STARTS HERE##########################################

In [2]:
C = ry.Config()
C.addFile('cargobot.g')
C.addFile('maze.g')
C.addFile('cargo.g')


qHome = C.getJointState()
C.setJointState(qHome)

Get Final Pose with KOMO

In [3]:
komo = ry.KOMO(C, 1, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.positionDiff, ['base', 'cargo_handle'], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e2], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")


{ time: 0.000796, evals: 9, done: 1, feasible: 0, sos: 0.252601, f: 0, ineq: 0, eq: 142 }
size of path: (1, 11)


0

RRT

In [4]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

for i in range(0, 2):
    ret = rrt.solve()
    print("rettttt: ", ret)
    path = ret.x
    print("path size:", path.shape[0])
    # display the path
    for t in range(0, path.shape[0] - 1):
        C.setJointState(path[t])
        C.view()
        time.sleep(0.002)

rettttt:  { time: 0.000786, evals: 8, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
path size: 14
rettttt:  { time: 0.000972, evals: 9, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
path size: 15


Trajectory Optimization via KOMO

In [5]:
C.setJointState(qHome)
komo2 = ry.KOMO(C, 1, path.shape[0], 2, True)
komo2.addControlObjective([], 0, 1e-1)
komo2.addControlObjective([], 2, 1e0)
komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)
komo2.initWithPath_qOrg(path)

ret = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
q = komo2.getPath()
print('last path:', q)

# display the path for part 1
for i in range(0, q.shape[0]):
    C.setJointState(q[i])
    C.view(False, f'waypoints{i}')
    time.sleep(0.02)


last path: [[-3.58299417e-02 -1.40722008e-02  2.34135930e-03 -1.34987795e-02
  -5.11737376e-01  8.07180511e-03 -2.01147289e+00 -9.42658000e-03
   1.99760336e+00 -5.06368853e-01  4.00000000e-02]
 [-3.88726353e-02 -6.55662504e-02  6.98031590e-03  6.78723530e-03
  -5.08666626e-01 -1.83776854e-02 -2.00218714e+00 -1.02606891e-02
   1.99633925e+00 -4.86104093e-01  3.94234079e-02]
 [-7.03034843e-02 -1.34661174e-01  1.61162089e-02 -9.49519545e-04
  -5.04272405e-01 -6.36943859e-02 -1.99945887e+00 -1.39668076e-02
   1.97952645e+00 -5.02170880e-01  3.96536730e-02]
 [-1.29873541e-01 -2.14886441e-01  2.99165502e-02  2.34093106e-02
  -5.39640482e-01 -7.20612295e-02 -1.98653303e+00 -4.97988123e-03
   2.00110753e+00 -5.09027014e-01  3.99333587e-02]
 [-1.96406771e-01 -3.01317506e-01  5.26492656e-02 -3.17686796e-03
  -5.46852690e-01 -1.16530774e-01 -1.99244205e+00 -1.45857837e-02
   1.98983092e+00 -4.93200507e-01  4.00000000e-02]
 [-2.92817151e-01 -3.81959423e-01  6.71806781e-02  1.57977971e-02
  -5.358

###########################################PART-2 ENDS HERE##########################################